<a href="https://colab.research.google.com/github/bogdanlalu/tensorflow/blob/master/NLP/02_Parsing_special_chars_and_stop_words.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2019 The TensorFlow Authors.

In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [1]:
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/bbc-text.csv \
    -O /tmp/bbc-text.csv

  
import csv
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


#Stopwords list from https://github.com/Yoast/YoastSEO.js/blob/develop/src/config/stopwords.js
# Convert it to a Python lis t and paste it here
stopwords = [ "a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "as", "at", "be", "because", "been", "before", "being", "below", "between", "both", "but", "by", "could", "did", "do", "does", "doing", "down", "during", "each", "few", "for", "from", "further", "had", "has", "have", "having", "he", "he'd", "he'll", "he's", "her", "here", "here's", "hers", "herself", "him", "himself", "his", "how", "how's", "i", "i'd", "i'll", "i'm", "i've", "if", "in", "into", "is", "it", "it's", "its", "itself", "let's", "me", "more", "most", "my", "myself", "nor", "of", "on", "once", "only", "or", "other", "ought", "our", "ours", "ourselves", "out", "over", "own", "same", "she", "she'd", "she'll", "she's", "should", "so", "some", "such", "than", "that", "that's", "the", "their", "theirs", "them", "themselves", "then", "there", "there's", "these", "they", "they'd", "they'll", "they're", "they've", "this", "those", "through", "to", "too", "under", "until", "up", "very", "was", "we", "we'd", "we'll", "we're", "we've", "were", "what", "what's", "when", "when's", "where", "where's", "which", "while", "who", "who's", "whom", "why", "why's", "with", "would", "you", "you'd", "you'll", "you're", "you've", "your", "yours", "yourself", "yourselves" ]

--2020-05-12 08:26:52--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/bbc-text.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.204.128, 2404:6800:4008:c07::80
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.204.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5057493 (4.8M) [application/octet-stream]
Saving to: ‘/tmp/bbc-text.csv’

/tmp/bbc-text.csv   100%[===================>]   4.82M  23.6MB/s    in 0.2s    

2020-05-12 08:26:52 (23.6 MB/s) - ‘/tmp/bbc-text.csv’ saved [5057493/5057493]



In [2]:
len(stopwords)

153

In [3]:
!wget --no-check-certificate \
    http://mlg.ucd.ie/files/datasets/stopwords.txt \
    -O /tmp/stopwords2.txt

--2020-05-12 08:27:00--  http://mlg.ucd.ie/files/datasets/stopwords.txt
Resolving mlg.ucd.ie (mlg.ucd.ie)... 137.43.93.132
Connecting to mlg.ucd.ie (mlg.ucd.ie)|137.43.93.132|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1780 (1.7K) [text/plain]
Saving to: ‘/tmp/stopwords2.txt’

/tmp/stopwords2.txt 100%[===================>]   1.74K  --.-KB/s    in 0s      

2020-05-12 08:27:01 (189 MB/s) - ‘/tmp/stopwords2.txt’ saved [1780/1780]



In [4]:
!ls -lh /tmp

total 4.9M
-rw-r--r-- 1 root root 4.9M Apr 26  2019 bbc-text.csv
-rw-r--r-- 1 root root 1.8K Jan 31  2014 stopwords2.txt


In [0]:
with open('/tmp/stopwords2.txt', 'r') as f:
  words = f.readlines()

stopwords2 = [w.replace('\n', '') for w in words]

In [6]:
len(stopwords2)

300

189 extra stopwords in the second list. The first 10 listed below

In [22]:
[word for word in stopwords2 if word not in stopwords][:10]

['according',
 'across',
 'actually',
 'adj',
 'afterwards',
 'almost',
 'along',
 'already',
 'also',
 'although']

In [23]:
!head /tmp/bbc-text.csv

category,text
tech,tv future in the hands of viewers with home theatre systems  plasma high-definition tvs  and digital video recorders moving into the living room  the way people watch tv will be radically different in five years  time.  that is according to an expert panel which gathered at the annual consumer electronics show in las vegas to discuss how these new technologies will impact one of our favourite pastimes. with the us leading the trend  programmes and other content will be delivered to viewers via home networks  through cable  satellite  telecoms companies  and broadband service providers to front rooms and portable devices.  one of the most talked-about technologies of ces has been digital and personal video recorders (dvr and pvr). these set-top boxes  like the us s tivo and the uk s sky+ system  allow people to record  store  play  pause and forward wind tv programmes when they want.  essentially  the technology allows for much more personalised tv. they are also bein

Read data from csv file

In [8]:
sentences = []
labels = []
with open("/tmp/bbc-text.csv", 'r') as csvfile:
  reader = csv.DictReader(csvfile)
  for row in reader:
    sentence = row['text']
    sentences.append(row['text'])
    labels.append(row['category'])

print(len(sentences))
print(sentences[0])

#Expected output
# 2225
# tv future hands viewers home theatre systems plasma high-definition tvs digital video recorders moving living room way people watch tv will radically different five years time. according expert panel gathered annual consumer electronics show las vegas discuss new technologies will impact one favourite pastimes. us leading trend programmes content will delivered viewers via home networks cable satellite telecoms companies broadband service providers front rooms portable devices. one talked-about technologies ces digital personal video recorders (dvr pvr). set-top boxes like us s tivo uk s sky+ system allow people record store play pause forward wind tv programmes want. essentially technology allows much personalised tv. also built-in high-definition tv sets big business japan us slower take off europe lack high-definition programming. not can people forward wind adverts can also forget abiding network channel schedules putting together a-la-carte entertainment. us networks cable satellite companies worried means terms advertising revenues well brand identity viewer loyalty channels. although us leads technology moment also concern raised europe particularly growing uptake services like sky+. happens today will see nine months years time uk adam hume bbc broadcast s futurologist told bbc news website. likes bbc no issues lost advertising revenue yet. pressing issue moment commercial uk broadcasters brand loyalty important everyone. will talking content brands rather network brands said tim hanlon brand communications firm starcom mediavest. reality broadband connections anybody can producer content. added: challenge now hard promote programme much choice. means said stacey jolna senior vice president tv guide tv group way people find content want watch simplified tv viewers. means networks us terms channels take leaf google s book search engine future instead scheduler help people find want watch. kind channel model might work younger ipod generation used taking control gadgets play them. might not suit everyone panel recognised. older generations comfortable familiar schedules channel brands know getting. perhaps not want much choice put hands mr hanlon suggested. end kids just diapers pushing buttons already - everything possible available said mr hanlon. ultimately consumer will tell market want. 50 000 new gadgets technologies showcased ces many enhancing tv-watching experience. high-definition tv sets everywhere many new models lcd (liquid crystal display) tvs launched dvr capability built instead external boxes. one example launched show humax s 26-inch lcd tv 80-hour tivo dvr dvd recorder. one us s biggest satellite tv companies directtv even launched branded dvr show 100-hours recording capability instant replay search function. set can pause rewind tv 90 hours. microsoft chief bill gates announced pre-show keynote speech partnership tivo called tivotogo means people can play recorded programmes windows pcs mobile devices. reflect increasing trend freeing multimedia people can watch want want.

2225
tv future in the hands of viewers with home theatre systems  plasma high-definition tvs  and digital video recorders moving into the living room  the way people watch tv will be radically different in five years  time.  that is according to an expert panel which gathered at the annual consumer electronics show in las vegas to discuss how these new technologies will impact one of our favourite pastimes. with the us leading the trend  programmes and other content will be delivered to viewers via home networks  through cable  satellite  telecoms companies  and broadband service providers to front rooms and portable devices.  one of the most talked-about technologies of ces has been digital and personal video recorders (dvr and pvr). these set-top boxes  like the us s tivo and the uk s sky+ system  allow people to record  store  play  pause and forward wind tv programmes when they want.  essentially  the technology allows for much more personalised tv. they are also being built-in to 

Remove special characters and stopwords from the sentences

In [0]:
def clean_chars(sentences):
  """Removes special characters from a list of sentences."""
  special_chars = r'.!@£$%^&*()_+"={}[]:"|;\<>?,./`~`'
  parsed_sentences = []

  # go through each sentence
  # go through each char
  # check if char is not part of special_chars and if so, append it to a list of clean chars
  # join this list of clean chars to create words
  # append this list of words (clean sentence) to the list of new sentences which will be returned

  for sentence in sentences:
    clean_chars = []

    for char in sentence:
      if char not in special_chars:
        clean_chars.append(char)
  
    clean_sentence = ''.join(clean_chars)  # space is already a clean char
    parsed_sentences.append(clean_sentence)
  
  return parsed_sentences

In [0]:
def remove_stop_words(sentences, stopwords=['']):
  parsed_sentences = []

  for sentence in sentences:
    parsed_words = []

    for word in sentence.split(' '):
      if word.lower() not in stopwords:
        parsed_words.append(word.lower())

    parsed_sentence = ' '.join(parsed_words)
    parsed_sentence = parsed_sentence.replace ("  ", " ")
    parsed_sentences.append(parsed_sentence)

  return parsed_sentences


In [0]:
def get_num_words(corpus):
  sum = 0
  for document in corpus:
    sum += len(document.split(' '))
  print(sum)

In [12]:
get_num_words(sentences)

933960


In [0]:
sentences_clean_chars = clean_chars(sentences)
sentences = remove_stop_words(sentences_clean_chars, stopwords=stopwords2)

In [14]:
get_num_words(sentences)

476726


In [29]:
tokenizer = Tokenizer(oov_token='<OOV>')
tokenizer.fit_on_texts(sentences)
word_index = tokenizer.word_index
print(len(word_index))
# Expected output
# 29714

30007


Helper functions

In [0]:
def getkey(d, id):
  """Returns the Key for the given value"""
  for pair in d.items():
    if pair[1] == id:
      print(pair[0])

def getval(d, key):
  """Returns the Value for the given"""
  print(d.get(key))

In [31]:
getval(word_index,'made')

7362


In [32]:
sequences = tokenizer.texts_to_sequences(sentences)
padded = pad_sequences(sequences, padding='post')
print(padded[0])
print(padded.shape)

[  70  139 1065 ...    0    0    0]
(2225, 2085)


In [18]:
label_tokenizer = Tokenizer()
label_tokenizer.fit_on_texts(labels)
label_word_index = label_tokenizer.word_index
label_seq = label_tokenizer.texts_to_sequences(labels)

print(label_seq)
print(label_word_index)

[[4], [2], [1], [1], [5], [3], [3], [1], [1], [5], [5], [2], [2], [3], [1], [2], [3], [1], [2], [4], [4], [4], [1], [1], [4], [1], [5], [4], [3], [5], [3], [4], [5], [5], [2], [3], [4], [5], [3], [2], [3], [1], [2], [1], [4], [5], [3], [3], [3], [2], [1], [3], [2], [2], [1], [3], [2], [1], [1], [2], [2], [1], [2], [1], [2], [4], [2], [5], [4], [2], [3], [2], [3], [1], [2], [4], [2], [1], [1], [2], [2], [1], [3], [2], [5], [3], [3], [2], [5], [2], [1], [1], [3], [1], [3], [1], [2], [1], [2], [5], [5], [1], [2], [3], [3], [4], [1], [5], [1], [4], [2], [5], [1], [5], [1], [5], [5], [3], [1], [1], [5], [3], [2], [4], [2], [2], [4], [1], [3], [1], [4], [5], [1], [2], [2], [4], [5], [4], [1], [2], [2], [2], [4], [1], [4], [2], [1], [5], [1], [4], [1], [4], [3], [2], [4], [5], [1], [2], [3], [2], [5], [3], [3], [5], [3], [2], [5], [3], [3], [5], [3], [1], [2], [3], [3], [2], [5], [1], [2], [2], [1], [4], [1], [4], [4], [1], [2], [1], [3], [5], [3], [2], [3], [2], [4], [3], [5], [3], [4], [2],